# Heritability and Genetic Correlation

https://github.com/bulik/ldsc/wiki/Heritability-and-Genetic-Correlation

In [ ]:
import re
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.stats import multitest
import warnings
warnings.filterwarnings('ignore')

In [ ]:
OUTCOME = 'T1D_STRICT'
data_dir = os.getcwd()+'/data/'
# res_dir = os.getcwd()+'/res/'
res_dir = '/Users/feiwang/Documents/Materials/t1d/'

In [ ]:
eps = ['CHIRBIL_PRIM','D3_AIHA_OTHER','D3_ALLERGPURPURA','D3_ANAEMIA_B12_DEF','D3_ITP','D3_SARCOIDOSIS','E4_ADDISON','E4_GRAVES_STRICT',
       'E4_HYTHY_AI_STRICT','G6_GUILBAR','G6_MS','G6_MYASTHENIA','K11_COELIAC','K11_IBD','L12_ALOPECAREATA','L12_PSORIASIS',
       'L12_VITILIGO','M13_ANKYLOSPON','M13_MCTD','M13_RHEUMA','M13_SJOGREN','M13_SYSTSLCE','M13_WEGENER','N14_IGA_NEPHROPATHY','SLE_FG',
       'T1D_STRICT']
eps_dict = {
    'D3_AIHA_OTHER':'Autoimmune hemolytic anemia',
    'D3_ALLERGPURPURA':'Allergic purpura',
    'D3_ANAEMIA_B12_DEF':'Vitamin B12 deficiency anaemia',
    'D3_ITP':'Idiopathic thrombocytopenic purpura',
    'D3_SARCOIDOSIS': 'Sarcoidosis',
    'CHIRBIL_PRIM':'Primary biliary cholangitis',
    'K11_COELIAC':'Coeliac disease',
    'K11_IBD':'Inflammatory bowel disease',
    'N14_IGA_NEPHROPATHY':'IgA nephropathy',
    'M13_ANKYLOSPON': 'Ankylosing spondylitis',
    'M13_MCTD':'Mixed connective tissue disease',
    'M13_RHEUMA':'Rheumatoid arthritis',
    'M13_SJOGREN':'Sjögren syndrome',
    'M13_SYSTSLCE':'Systemic sclerosis',
    'M13_WEGENER':'Wegener granulomatosis',
    'SLE_FG':'Systemic lupus erythematosus',
    'G6_GUILBAR':'Guillain-Barre syndrome',
    'G6_MS':'Multiple Sclerosis',
    'G6_MYASTHENIA':'Myasthenia gravis',
    'L12_ALOPECAREATA':'Alopecia areata',
    'L12_PSORIASIS':'Psoriasis',
    'L12_VITILIGO':'Vitiligo',
    'E4_ADDISON':'Adrenocortical insufficiency',
    'E4_GRAVES_STRICT':'Autoimmune hyperthyroidism',
    'E4_HYTHY_AI_STRICT':'Autoimmune hypothyroidism',
    'T1D_STRICT':'Type 1 diabetes'
}

## 1. Process data for LDSC
check if ref allele and alt allele are on the same direction.

### if good summary stats can be found from GWAS Catalog:
https://www.ebi.ac.uk/gwas/

In [ ]:
# Choose the summary stats by the effect size. The bigger, the better.
# formula: ((4 * n_cases * n_controls)/(n_cases + n_controls))

ep = 'SLE_FG'
# 'T1D_STRICT', 'K11_COELIAC', 'SLE_FG', 'M13_RHEUMA', 'T2D', 'L12_VITILIGO'
# 'G6_MYASTHENIA', 'J10_ASTHMA', 'G6_MS', 'L12_PSORIASIS'

file_in = res_dir+'data/'+ep+'.txt' # tsv txt
# file_in = res_dir+'data/'+ep+'_meta_out.tsv.gz'
file_out = res_dir+'processed_stats_/'+ep+'.premunge.gz'
if not os.path.isfile(file_in):
    file_in = res_dir+'data/'+ep+'.tsv'
if not os.path.isfile(file_in):
    file_in = res_dir+'data/'+ep+'.csv'
if not os.path.isfile(file_in):
    print('Data cannot be found!')
df = pd.read_csv(file_in, sep='\t')
if len(df.columns) == 1:
    df = pd.read_csv(file_in, sep=' ')
if len(df.columns) == 1:
    df = pd.read_csv(file_in)
#     try:
#         df = pd.read_csv(file_in, sep=' ')
#     except pd.errors.ParserError:
#         df = pd.read_csv(file_in)
df = df.rename(columns={
    'rsid': 'snpid', 'SNPID': 'snpid', 'variant_id': 'snpid', 'SNP': 'snpid',
    'effect_allele': 'a1', 'A1': 'a1', 'A1_effect': 'a1',
    'other_allele': 'a2', 'A2': 'a2', 'A2_other': 'a2',
    'p': 'pval', 'P-val': 'pval', 'p_value': 'pval', 'P_EUR': 'pval',
    'OR(A1)': 'or', 'odds_ratio': 'or',
    'standard_error': 'se', 'se_EUR': 'se',
    'beta_EUR': 'beta'
})
print(len(df))
df.head(3)

### if RSID is not in the summary stats:

In [ ]:
Infile=res_dir+'data/'+"L12_PSORIASIS.tsv"
OutFile=res_dir+'data/'+"L12_PSORIASIS_TEST.tsv"
HM3ref=res_dir+'data/'+"HM3Ref"
build=res_dir+'data/'+"hg37"

In [ ]:
df = pd.read_table(Infile, low_memory=False)
df['hg37'] = 'chr'+df.chromosome.astype(str)+':'+df.base_pair_location.astype(str)
df['A1'] = df.effect_allele.str.upper()
df['A2'] = df.other_allele.str.upper()
Ref = pd.read_table(HM3ref)
ref = Ref.rename(columns={'REF': 'A1', 'ALT': 'A2'})
tmp1 = pd.merge(df, ref, how = 'inner', on = [build, 'A1', 'A2'])
tmp1 = tmp1.drop(['hg36', 'hg37', 'hg38'], axis=1)
ref = Ref.rename(columns={'REF': 'A2', 'ALT': 'A1'})
tmp2 = pd.merge(df, ref, how = 'inner', on = [build, 'A1', 'A2'])
tmp2 = tmp2.drop(['hg36', 'hg37', 'hg38'], axis=1)
tmp = pd.concat([tmp1, tmp2])
tmp.to_csv(OutFile, sep = '\t', index = None)

In [ ]:
# All SNPs on chromosome 23 have been removed from the list
Ref['chr'] = Ref.hg38.str.extract('chr(\d+)\:')
Ref[Ref.chr=='23']

### if an endpoint has sub dataset for each chromosome:

In [ ]:
df = pd.read_csv(data_dir+'/vitiligo/GWAS123chrXcmh.txt', sep='\t')
for i in tqdm.tqdm(range(1, 23)):
    df_ = pd.read_csv(data_dir+'/vitiligo/GWAS123chr'+str(i)+'cmh.txt', sep='\t')
    df_ = df_.rename(columns={'CMH P':'P', 'ORX':'OR'})
    df = pd.concat([df, df_], axis=0)
df['snpid'] = df.SNP.str.lower()
df = df.rename(columns={'A1': 'a1', 'A2': 'a2', 'P': 'p', 'OR': 'or', 'SE': 'se'})
df = df[['a1', 'a2', 'p', 'or', 'se', 'snpid']]
df.to_csv(data_dir+'/autoimmune_gwas_sumstats/L12_VITILIGO.txt', index=None)

### if the format of the summary stats has a problem:
the number of the cols in some rows are larger than the number of the header

In [ ]:
# 'K11_IBD', 'K11_CROHN', 'M13_SJOGREN' finn, 'M13_SYSTSLCE' 格式, 'G6_MS'

ep = 'K11_CROHN'
file_in = data_dir+ep+'.txt'
file_out = data_dir+ep+'.premunge.gz'
matrix = []
with open(file_in) as f:
    for line in f:
        row = line.split(' ')
        if len(row) != 8:
            row = row[:7] + [row[-1]]
        matrix.append(row)
matrix = np.array(matrix)
df = pd.DataFrame(matrix[1:], columns=matrix[0])
df['snpid'] = df['SNP\n'].str[:-1]
df = df.rename(columns={
    'Allele1': 'a1',
    'Allele2': 'a2',
    'P.value': 'pval',
    'Effect': 'beta'
})
df['beta'] = df.beta.astype(float)
df['pval'] = df.pval.astype(float)
print(len(df))
df.head(3)


In [ ]:
# find out which paper using this data by searching the newly found loci in the paper
rsid = 'rs80244186'
a=df[df.snpid == rsid]
print('or: ', round(np.exp(a.iloc[0,-1]), 2))
print('p:  ', a.iloc[0,-2])

### if summary stats can be find from Finngen results:
1. to download data from Finngen + UKBB:
- https://finngen.gitbook.io/finngen-analyst-handbook/finngen-data-specifics/green-library-data-aggregate-data/other-analyses-available/meta-analysis-finngen-ukbb-estbbuntitled
- gsutil cp gs://finngen-production-library-green/finngen_R9/finngen_R9_analysis_data/ukbb_meta/meta/G6_MS_meta_out.tsv.gz /Users/feiwang/Documents/Materials/familial_trajectory/data

In [ ]:
# L12_ALOPECAREATA, N14_IGA_NEPHROPATHY, GEST_DIABETES, G6_OTHDEMYEL, E4_THYROIDITAUTOIM,
# E4_HYTHY_AI_STRICT, D3_ANAEMIA_B12_DEF, CHIRBIL_PRIM, AUTOIMMUNE_HYPERTHYROIDISM, M13_SJOGREN

ep = 'G6_MS'
file_in = res_dir+'data/'+ep+'_meta_out.tsv.gz'
file_out = res_dir+'processed_stats/'+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
df = df.rename(columns={
    'rsid': 'snpid',
    'REF': 'a1',
    'ALT': 'a2',
    'all_inv_var_meta_p': 'pval',
    'all_inv_var_meta_beta': 'beta'
})
print(len(df))
df.head(3)

2. to download data from Finngen if the endpoint cannot be found from Finngen + UKBB:
- gsutil cp gs://finngen-production-library-green/finngen_R9/finngen_R9_analysis_data/summary_stats/release/finngen_R9_M13_RHEUMA.gz /Users/feiwang/Documents/Materials/familial_trajectory/data

In [ ]:
# D3_AIHA_OTHER, D3_ALLERGPURPURA, D3_ITP, E4_ADDISON, G6_DISSOTH, G6_GUILBAR,
# H7_IRIDOCYC_ANTER, L12_DERMATHERP, M13_MCTD, M13_WEGENER

ep = 'K11_COELIAC'
file_in = res_dir+'data/finngen_R9_'+ep+'.gz'
file_out = res_dir+'processed_stats/'+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
df = df.rename(columns={
    'rsids': 'snpid',
    'ref': 'a1',
    'alt': 'a2',
})
print(len(df))
df.head(3)

### save the data as a pre-munge file

In [ ]:
# if the data above looks all good, save it as a zipped tsv
df = df[['snpid', 'A2', 'A1', 'pval', 'beta']]
df.to_csv(file_out, sep='\t', compression='gzip', index=None)

In [ ]:
df = df.rename(columns={'a1':'A2', 'a2':'A1'})
df.head(3)

## 2. Munge the data and apply LDSC in terminal
- In terminal, run code below:
<code>
    cd Projects/familial_trajectory/ldsc
    source activate ldsc
    ep1=T1D_STRICT
    ep2=K11_IBD
    python munge_sumstats.py \
        --sumstats data/$ep2.premunge.gz \
        --N-cas 2051 \
        --N-con 594747 \
        --out data/$ep2 \
        --merge-alleles w_hm3.snplist \
        --chunksize 500000
    python ldsc.py \
        --rg data/$ep1.sumstats.gz,data/$ep2.sumstats.gz \
        --ref-ld-chr eur_w_ld_chr/ \
        --w-ld-chr eur_w_ld_chr/ \
        --out res/$ep1.$ep2
</code>

## 3. Extract the results from log files and merge the information
https://academic.oup.com/bioinformatics/article/33/2/272/2525718
- Heritability (H2) Z score is at least > 1.5 (optimal > 4)
- Mean Chi square of the test statistics > 1.02
- The intercept estimated from the SNP heritability analysis is between 0.9 and 1.1

In [ ]:
header1 = ['mean chi2_1', 'b_1', 'mean chi2_2', 'b_2']
header2 = ['ep1', 'ep2', 'rg', 'se','z', 'p', 'h2_obs', 'h2_obs_se', 'h2_int', 'h2_int_se', 'gcov_int', 'gcov_int_se']

def findStats(string):
    stats1 = re.findall(r'data/([\w_\d]+)\.sumstats\.gz', string)
    stats2 = re.findall(r'\s+([-]{0,1}\d+\.\d+(e\-\d+){0,1})', string)
    if len(stats2) != 10:
        print(string)
    stats2 = [float("{:.4f}".format(float(i[0]))) for i in stats2]
    stat_dict = dict(zip(header2, stats1+stats2))
    return stat_dict

def readLog(log_file, df):
    with open(log_file) as f:
        log = f.readlines()
    if len(log) != 0:
        findings1 = []
        for line in log:
            if re.match(r'^Mean Chi\^2\: ', line): 
                findings1.append(re.findall(r'^Mean Chi\^2\: (.+)$', line)[0])
            if re.match(r'^Intercept\: ', line): 
                findings1.append(re.findall(r'^Intercept\: (.+) \(\d', line)[0])
            if re.match(r'^data/', line):
                findings2 = findStats(line)
        findings1 = dict(zip(header1, findings1))
        df = df.append(pd.Series({**findings1, **findings2}), ignore_index=True)
    return df

In [ ]:
ep1 = 'T1D_STRICT'
stat_df = pd.DataFrame(columns=header1+header2, dtype=object)
for ep2 in eps[1:]:
    f_name = res_dir+'ldsc_res/'+ep1+'.'+ep2+'.log'
    if os.path.isfile(f_name):
        stat_df = readLog(f_name, stat_df)
    else:
        print(ep2)
stat_df = stat_df.sort_values(by='h2_obs')
stat_df['fdr_ldsc'], _ = multitest.fdrcorrection(stat_df.p)

### 3.1 Analysis without chromosome 6
Removal of HLA region in LDSC https://groups.google.com/g/ldsc_users/c/fEDtVvcm5oc

In [ ]:
# analysis without chromosome 6
hm3 = pd.read_table(res_dir+'w_hm3.snplist')
hm3 = hm3.merge(Ref[['SNP','hg38']], 'left', on='SNP')
hm3['chr'] = hm3.hg38.str.extract('^(chr\d+):')
hm3_without6 = hm3[hm3.chr != 'chr6']
hm3_without6.to_csv(res_dir+'hm3_without6.csv', index=None)

In [ ]:
ep = 'M13_RHEUMA'
file_in = res_dir+'processed_stats_/'+ep+'.premunge.gz'
file_out = res_dir+'processed_stats_without6/'+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
print(len(df))
df.head(3)

In [ ]:
df = df[df.snpid.isin(hm3_without6.SNP)]
print(len(df))
df.to_csv(file_out, sep='\t', compression='gzip', index=None)

In [ ]:
res_without6 = [
    'data/T1D_STRICT.sumstats.gz  data/M13_RHEUMA.sumstats.gz 0.5217  0.1173 4.4488  8.6334e-06   0.815     0.2718  1.1445     0.0642    0.173       0.0663',
    'data/T1D_STRICT.sumstats.gz  data/L12_DERMATHERP.sumstats.gz  0.4335  0.312  1.3895  0.1647   0.001     0.0012  1.0086     0.0073    0.0032       0.0051',
    'data/T1D_STRICT.sumstats.gz  data/AUTOIMMUNE_HYPERTHYROIDISM.sumstats.gz  0.4749  0.0929  5.1124  3.1812e-07  0.0047     0.0009  1.0076     0.0071    0.0024       0.0064',
    'data/T1D_STRICT.sumstats.gz  data/D3_ANAEMIA_B12_DEF.sumstats.gz  0.5353  0.0958  5.5863  2.3189e-08   0.004     0.0008  1.0294     0.0078    0.0111       0.0075',
    'data/T1D_STRICT.sumstats.gz  data/K11_COELIAC.sumstats.gz 0.252  0.0729 3.4578  0.0005  0.2432     0.0483  1.0626     0.0087   0.1115       0.0067',
    'data/T1D_STRICT.sumstats.gz  data/E4_HYTHY_AI_STRICT.sumstats.gz  0.4592  0.0435  10.5483  5.1703e-26  0.0626     0.0072  1.1368     0.0286    0.0258       0.0119'
]
stat_df_without6 = pd.DataFrame(columns=header2, dtype=object)
for i in res_without6:
    stat_df_without6 = stat_df_without6.append(pd.Series(findStats(i)), ignore_index=True)
    
stat_compare = stat_df_without6[['ep1', 'ep2', 'rg', 'p']].rename(columns={'rg':'rg_w/o6', 'p':'p_w/o6'})
stat_compare = stat_compare.merge(stat_df[['ep2','rg','p']].rename(columns={'rg':'rg_w6', 'p':'p_w6'}), 'left', on='ep2')
stat_compare = stat_compare[['ep1', 'ep2', 'rg_w/o6', 'rg_w6', 'p_w/o6', 'p_w6']]

In [ ]:
stat_compare

## 4. convert h2 from observed scale to liability scale
- https://gist.github.com/nievergeltlab/fb8a20feded72030907a9b4e81d1c6ea
- https://www.sciencedirect.com/science/article/pii/S0002929711000206?via%3Dihub

In [ ]:
# Method 1:
import json
with open('/Users/feiwang/Documents/Projects/stats__2021-03-25.json') as f:
    stats = json.load(f)
stat_df['prevalence'] = [stats['stats'][i]['prevalence_all'] for i in stat_df.ep2]

In [ ]:
# prevalence of T1D in FinRegistry
stats['stats']['T1D_STRICT']['prevalence_all']

In [ ]:
# Method 2:
stats = pd.read_excel(res_dir+'summary_of_ADs.xlsx', sheet_name='summary')[['Endpoint', 'Prevalence', 'case', 'control']]
stat_df = stat_df.merge(stats.rename(columns={'Endpoint':'ep2', 'Prevalence':'prevalence'}), 'left', on='ep2') 
stat_df['prevalence'] = stat_df['prevalence']*0.01

h2_liab <- h2 * K^2 * ( 1 - K)^2 / P / (1-P) / zv^2

var_h2_liab <- ( seh2 * K^2 * ( 1 - K)^2 / P / (1-P) / zv^2) ^2

In [ ]:
stat_df['z_pdf'] = norm.pdf(norm.ppf(1-stat_df['prevalence']))
stat_df['proportion'] = stat_df.case / stat_df.control
stat_df['h2_lia'] = stat_df.h2_obs*(stat_df.prevalence**2)*(1-stat_df.prevalence)**2/stat_df.proportion/(1-stat_df.proportion)/(stat_df.z_pdf**2)
stat_df['se_lia'] = stat_df.h2_obs_se*(stat_df.prevalence**2)*(1-stat_df.prevalence)**2/stat_df.proportion/(1-stat_df.proportion)/(stat_df.z_pdf**2)

In [ ]:
from scipy.stats import chi2
chi2.cdf(1-(0.021359**2)/(0.046990**2), 1)

In [ ]:
stat_df['rg_025'] = stat_df.rg - 1.96*stat_df.se
stat_df['rg_975'] = stat_df.rg + 1.96*stat_df.se

In [ ]:
stat_df.sort_values('ep2')